# Example 5-3: Finding the Moon Position Vector
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, pp. 294-295

This notebook demonstrates calculating the geocentric position of the Moon given the date, time, and correctional parameters.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy==0.4.1

Import `numpy` and the relevant `valladopy` modules:

In [2]:
import numpy as np
from valladopy.astro.celestial import moon
from valladopy.mathtime.julian_date import convtime

## Problem Definition
---

GIVEN: &ensp;April 27, 1994, 23:58:59.816 UTC<br>
&emsp;&emsp;&emsp;&emsp;$\Delta UT1 = -0.088972$s and $\Delta AT = 28$s<br>
FIND: &emsp;Geocentric position vector of the Moon

In [3]:
year = 1994
month = 4
day = 27
hour = 23
minute = 58
second = 59.816
timezone = 0  # hours offset from UTC

dut1 = -0.088972
dat = 28

## Solution
---

We begin by finding the Julian Date using the `convtime` routine (see [Example 3-7](Example_3-7.ipynb) for more details on this process), since we are also given the leap seconds and $\Delta UT1$:

In [4]:
_, _, jd, jdfrac, *_ = convtime(year, month, day, hour, minute, second, timezone, dut1, dat)

print(f'Julian Date:\t{jd + jdfrac:.4f}')

Julian Date:	2449470.4993


**Algorithm 31** outlines the steps to find the Moon position vector. First find the Julian centuries of TDB (letting $TDB \cong TT$):

$$
T_{TDB} = \frac{JD_{TDB} - 2,451,545.0}{36,525}
$$

The ecliptic longitude of the Moon is:


$$
\begin{align*}
\lambda_{\text{ecliptic ☾}} &= 218.32^\circ + 481,\!267.8813\,T_{TDB} \\
&\quad + 6.29 \sin(134.9^\circ + 477,\!198.85\,T_{TDB}) \\
&\quad - 1.27 \sin(259.2^\circ - 413,\!335.38\,T_{TDB}) \\
&\quad + 0.66 \sin(235.7^\circ + 890,\!534.23\,T_{TDB}) \\
&\quad + 0.21 \sin(269.9^\circ + 954,\!397.70\,T_{TDB}) \\
&\quad - 0.19 \sin(357.5^\circ + 35,\!999.05\,T_{TDB}) \\
&\quad - 0.11 \sin(186.6^\circ + 966,\!404.05\,T_{TDB})
\end{align*}
$$

The ecliptic latitude of the Moon is:

$$
\begin{align*}
\phi_{\text{ecliptic } ☾} =\ &5.13^\circ \sin(93.3 + 483,\!202.03\,T_{TDB}) \\
&+ 0.28 \sin(228.2 + 960,\!400.87\,T_{TDB}) \\
&- 0.28 \sin(318.3 + 6003.18\,T_{TDB}) \\
&- 0.17 \sin(217.6 - 407,\!332.20\,T_{TDB})
\end{align*}
$$

The horizontal parallax is:

$$
\begin{align*}
\mathscr{P} &= 0.9508^\circ \\
&\quad + 0.0518 \cos(134.9^\circ + 477,\!198.85\,T_{TDB}) \\
&\quad + 0.0095 \cos(259.2^\circ - 413,\!335.38\,T_{TDB}) \\
&\quad + 0.0078 \cos(235.7^\circ + 890,\!534.23\,T_{TDB}) \\
&\quad + 0.0028 \cos(269.9^\circ + 954,\!397.70\,T_{TDB})
\end{align*}
$$

The obliquity of the ecliptic is:

$$
\epsilon = 23.439291^\circ - 0.0130042\,T_{TDB} - 1.64 \times 10^{-7}\,T_{TDB}^2 + 5.04 \times 10^{-7}\,T_{TDB}^3
$$

Next, find the magnitude of the position vector:

$$
r_☾ = \frac{R_{\oplus\ ☾}}{\sin(\mathscr{P})}
$$

Finally, the position vector is:

$$
\vec{r}_☾ = r_☾ \begin{bmatrix}
\cos(\phi_{ecliptic\ ☾})\cos(\lambda_{ecliptic\ ☾}) \\
\cos(\epsilon)\cos(\phi_{ecliptic\ ☾})\sin(\lambda_{ecliptic\ ☾}) - \sin(\epsilon)\sin(\phi_{ecliptic\ ☾}) \\
\sin(\epsilon)\cos(\phi_{ecliptic\ ☾})\sin(\lambda_{ecliptic\ ☾}) + \cos(\epsilon)\sin(\phi_{ecliptic\ ☾})
\end{bmatrix}
$$

We can determine the right ascension and declination using **Equation 4-20**:

$$
\begin{aligned}
\sin(\delta) &= \sin(\phi_{ecliptic\ ☾}) \cos(\epsilon) + \cos(\phi_{ecliptic\ ☾}) \sin(\epsilon) \sin(\lambda_{ecliptic\ ☾}) \\
\\
\sin(\alpha) &= \frac{-sin(\phi_{ecliptic\ ☾}) \sin(\epsilon) + \cos(\phi_{ecliptic\ ☾}) \cos(\epsilon) \sin(\lambda_{ecliptic\ ☾})}{cos(\delta)}
\end{aligned}
$$

Call the `moon.position` routine to accomplish this:

In [5]:
rmoon, rtasc, decl = moon.position(jd + jdfrac)

print(f'Moon position vector:\t{rmoon}\tkm')
print(f'Right ascension:\t{np.degrees(rtasc):.6}\tdeg')
print(f'Declination:\t\t{np.degrees(decl):.6}\tdeg')

Moon position vector:	[-134300.14875768 -311544.89187406 -126688.46735048]	km
Right ascension:	-113.32	deg
Declination:		-20.477	deg
